In [1]:
# Save the books

import requests

# Get a request and save it as a txt file

books = {
    "863": "The Mysterious Affair at Styles",
    "1155": "The Secret Adversary",
    "58866": "The Murder on the Links",
    "61168": "The Man in the Brown Suit",
    "61262": "Poirot Investigates",
    "65238": "The Secret of Chimneys",
    "66446": "The Plymouth Express Affair",
    "67160": "The Hunter's Lodge Case",
    "67173": "The Missing Will",
    "69087": "The Murder of Roger Ackroyd",
    "70114": "The Big Four",
    "72824": "The Mystery of the Blue Train"
}

# Go through the dict

for id, title in books.items():
    res = requests.get(f'http://www.gutenberg.org/ebooks/{id}.txt.utf-8')
    # Write to txt file

    cleaned_text = res.text.encode('utf-8', errors='replace').decode('utf-8')
    
    filename = "Data/" + title.replace(" ", "-") + ".txt"
    # with open(filename, 'w', encoding='utf-8') as file:
    #     # Clean res.text to have nothing that cannot be encoded in a .write() call
    #     file.write(cleaned_text)
    #     pass

In [18]:
import nltk
from nltk.tokenize import sent_tokenize

# Manually load the Punkt tokenizer

# Test sentence tokenization
text = "This is a test sentence. Let's see if it works."
print(sent_tokenize(text))

['This is a test sentence.', "Let's see if it works."]


In [19]:
# Python program to generate word vectors using Word2Vec

# importing all necessary modules
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action='ignore')


# Reads file
sample = open("Data/books/The-Mysterious-Affair-at-Styles.txt", encoding="utf-8")
s = sample.read()

# Replaces escape character with space
f = s.replace("\n", " ")

data = []

# iterate through each sentence in the file
for i in sent_tokenize(f):
	temp = []

	# tokenize and normalize the sentence into words
	for j in word_tokenize(i):
		temp.append(j.lower())

	data.append(temp)

# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count=1,
								vector_size=100, window=5)

# Print results
print("Cosine similarity between 'murder' " +
	"and 'investigate' - CBOW : ",
	model1.wv.similarity('murder', 'investigate'))

print("Cosine similarity between 'murder' " +
	"and 'said' - CBOW : ",
	model1.wv.similarity('murder', 'said'))

# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count=1, vector_size=100,
								window=5, sg=1)

# Print results
print("Cosine similarity between 'murder' " +
	"and 'investigate' - Skip Gram : ",
	model2.wv.similarity('murder', 'investigate'))

print("Cosine similarity between 'murder' " +
	"and 'said' - Skip Gram : ",
	model2.wv.similarity('murder', 'said'))


Cosine similarity between 'murder' and 'investigate' - CBOW :  0.9484985
Cosine similarity between 'murder' and 'said' - CBOW :  0.9735845
Cosine similarity between 'murder' and 'investigate' - Skip Gram :  0.9522923
Cosine similarity between 'murder' and 'said' - Skip Gram :  0.8787241


In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize, MWETokenizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re
from gensim.models import Word2Vec

# Make sure to download resources in separate file outside of Jupyter notebook
# nltk.download('punkt_tab') - pre-trained model for tokenization (sent_tokenize)
# nltk.download('stopwords') - removes common filler words like "the" "and" "is"
# nltk.download('wordnet') - database that groups words into sets of synoynms - used for lemmatization



# Reads file
Styles_file = open("Data/books/The-Mysterious-Affair-at-Styles.txt", encoding="utf-8")
Styles = Styles_file.read()

# Clean Text - Normalization - Lowercase, Remove Punctuation
Styles = Styles.lower()
#Styles = re.sub(r'[^\w\s]', '', Styles)

# Tokenize
sentences = sent_tokenize(Styles)
tokens = [word_tokenize(sentence) for sentence in sentences]

# Remove Stopwords and non alphanumeric characters
stop_words = set(stopwords.words('english'))
# tokens = [[word for word in sentence if word not in stop_words] for sentence in tokens]
tokens = [[word for word in sentence if word.isalnum() and word not in stop_words] for sentence in tokens]

#  Lemmatization (Optional)
#lemmatizer = WordNetLemmatizer()
# Part of Speech = Verb
#tokens = [[lemmatizer.lemmatize(word, pos='v') for word in sentence] for sentence in tokens]

model = Word2Vec(tokens, vector_size=100, window=5, min_count=1, sg=0) #CBOW model


words = list(model.wv.index_to_key) # Gets words from models vocabularly                             
w_vec = np.array([model.wv[word] for word in words]) # Word Vectors


# Training

model.build_vocab(tokens)
model.train(tokens, total_examples=model.corpus_count, epochs=5)


# Save Word Embedding
np.savetxt('./projector_files/trained0_embedding.tsv', w_vec, delimiter='\t')

# All Words in the novel saved
#with open('./projector_files/init_words_metadata.tsv', 'w') as f:
 #   for word in words:
  #      f.write(word + "\n")
        

## TRAINING 
#model.build_vocab(tokens)
#model.train(tokens, total_examples=model.corpus_count, epochs=5)


similar_words = model.wv.most_similar('murder', topn=5)
similar_words1 = model.wv.most_similar('poison', topn=5)
poison = model.wv.most_similar('strychnine',topn=5)

print(f"Words similar to murder: {similar_words}")
print(f"Words similar to poison: {similar_words1}")
print(f"Words similar to styrchnine: {poison}")



## Steps for visualizing
# 1. Install tensorflow - pip install tensorflow
# 2. Run Tensorboard - tensorboard --logdir=./projector_files/







Words similar to murder: [('last', 0.9989808201789856), ('strychnine', 0.9989655017852783), ('cynthia', 0.9988963007926941), ('cavendish', 0.9988837242126465), ('would', 0.9988781809806824)]
Words similar to poison: [('eyes', 0.9991862177848816), ('john', 0.9991543292999268), ('one', 0.9991313815116882), ('take', 0.9991292953491211), ('time', 0.9991012215614319)]
Words similar to styrchnine: [('inglethorp', 0.9997757077217102), ('cavendish', 0.9997653961181641), ('poirot', 0.9997626543045044), ('found', 0.9997562170028687), ('little', 0.9997479319572449)]
